In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# Model
class SimpleDepthEstimator(nn.Module):
    def __init__(self):
        super(SimpleDepthEstimator, self).__init__()
        # Use a pre-trained model as a feature extractor
        self.features = models.resnet18(pretrained=True)
        for param in self.features.parameters():
            param.requires_grad = False
        # Re-activate gradient updates for last layers
        for param in self.features.layer4.parameters():
            param.requires_grad = True
        self.features.fc = nn.Identity()  # Remove the classification head

        # Add custom layers for depth estimation
        self.depth_layers = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 1)  # Output a single depth value for simplicity
        )

    def forward(self, x):
        x = self.features(x)  # Extract features
        depth = self.depth_layers(x)  # Estimate depth
        return depth

In [3]:
# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [4]:
# Load all images from the dataset folder
data_folder = 'data22555'

In [5]:
image_filenames = os.listdir(data_folder)
images = []
for filename in image_filenames:
    img_path = os.path.join(data_folder, filename)
    image = Image.open(img_path)
    images.append(image)

In [6]:
# Split the data into training, validation, and testing sets
train_images, test_val_images = train_test_split(images, test_size=0.4, random_state=42)
val_images, test_images = train_test_split(test_val_images, test_size=0.5, random_state=42)


In [7]:
# Dataset class for tire images
class TireDataset(Dataset):
    """Basic Tire Image Dataset"""

    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]

        if self.transform:
            image = self.transform(image)
        else:
            transform = transforms.ToTensor()
            image = transform(image)

        return image

In [8]:
# Dataset and dataloader setup
train_dataset = TireDataset(train_images, transform=train_transform)
val_dataset = TireDataset(val_images, transform=val_transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


In [9]:
print(f"Training set size: {len(train_images)}")
print(f"Testing set size: {len(test_images)}")
print(f"Validation set size: {len(val_images)}")

Training set size: 50
Testing set size: 17
Validation set size: 17


In [10]:
# Instantiate the model
model = SimpleDepthEstimator()

c:\Users\free bytes\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\free bytes\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam([{'params': model.features.layer4.parameters(), 'lr': 1e-4},
                        {'params': model.depth_layers.parameters()}], lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [12]:
# Training loop
num_epochs = 25
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, torch.zeros_like(outputs))  # Dummy target for depth estimation

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, torch.zeros_like(outputs))  # Dummy target for depth estimation

            running_val_loss += loss.item() * inputs.size(0)

    epoch_val_loss = running_val_loss / len(val_loader.dataset)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')

    scheduler.step(epoch_val_loss)

Epoch [1/25], Train Loss: 0.1298, Val Loss: 0.0113


c:\Users\free bytes\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/25], Train Loss: 0.2090, Val Loss: 0.0135
Epoch [3/25], Train Loss: 0.1356, Val Loss: 0.0196
Epoch [4/25], Train Loss: 0.0981, Val Loss: 0.0311
Epoch [5/25], Train Loss: 0.0927, Val Loss: 0.0468
Epoch [6/25], Train Loss: 0.0838, Val Loss: 0.0857
Epoch [7/25], Train Loss: 0.0913, Val Loss: 0.0656
Epoch [8/25], Train Loss: 0.0865, Val Loss: 0.0333
Epoch [9/25], Train Loss: 0.0669, Val Loss: 0.0369
Epoch [10/25], Train Loss: 0.1088, Val Loss: 0.0149
Epoch [11/25], Train Loss: 0.0564, Val Loss: 0.0136
Epoch [12/25], Train Loss: 0.0601, Val Loss: 0.0302
Epoch [13/25], Train Loss: 0.0507, Val Loss: 0.0187
Epoch [14/25], Train Loss: 0.0349, Val Loss: 0.0152
Epoch [15/25], Train Loss: 0.0423, Val Loss: 0.0212
Epoch [16/25], Train Loss: 0.0294, Val Loss: 0.0198
Epoch [17/25], Train Loss: 0.0317, Val Loss: 0.0140
Epoch [18/25], Train Loss: 0.0256, Val Loss: 0.0171
Epoch [19/25], Train Loss: 0.0249, Val Loss: 0.0136
Epoch [20/25], Train Loss: 0.0268, Val Loss: 0.0168
Epoch [21/25], Train